In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn
import statsmodels.api as sm
import scipy as sc

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
df = pd.read_csv('train.csv')
df = df.set_index('id')
df.timestamp = pd.to_datetime(df.timestamp)
#df = df.fillna(df.mean())

In [3]:
df.head()

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
id,,,,,,,,,,,,,,,,,,,,,
1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
4,2011-01-09,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
5,2011-05-09,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [4]:
df_temp = pd.get_dummies(df)
df_temp.head()

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,water_1line_yes,big_road1_1line_no,big_road1_1line_yes,railroad_1line_no,railroad_1line_yes,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
id,,,,,,,,,,,,,,,,,,,,,
1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,1,0,0,1,0,0,0
2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,1,0,1,0,0,0,0
3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,1,0,0,0,0,1,0
4,2011-01-09,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,1,0,0,1,0,0,0
5,2011-05-09,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,1,1,0,0,0,0


In [5]:
cov = df_temp.corr()
cov_temp = cov[cov!=1].fillna(0)

In [6]:
temp_min ={w:(cov_temp[w].argmin(),cov_temp[w].min()) for w in cov_temp.index}
temp_max ={w:(cov_temp[w].argmax(),cov_temp[w].max()) for w in cov_temp.index}

In [7]:
temp_min['price_doc'],temp_max['price_doc']

(('zd_vokzaly_avto_km', -0.28406855884521937),
 ('num_room', 0.47633653800603459))

In [9]:
df_temp = df.dropna()
df_grouped = df.groupby(['sub_area'])
floor = df_temp.floor
price = df_temp.price_doc

In [10]:

'''
for name,group in df_grouped:
    plt.figure()
    plt.title(name)
    plt.scatter(group.max_floor,group.price_doc,label=name)

plt.legend()    
plt.show()
'''

'\nfor name,group in df_grouped:\n    plt.figure()\n    plt.title(name)\n    plt.scatter(group.max_floor,group.price_doc,label=name)\n\nplt.legend()    \nplt.show()\n'

In [11]:
from collections import *
dict_building = defaultdict()
l = lambda x:dict_building[x].append(df.price_doc)

In [12]:
df_temp['building'] = df_temp.build_year.map(int).map(str)+ df_temp.sub_area + df_temp.material.map(int).map(str)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
df_grouped1 = df_temp.groupby('building')['price_doc'].count()
df_grouped = df_temp.groupby('building')['price_doc'].mean()

In [14]:
df_grouped[df_grouped1[df_grouped1>10].index]

building
0Tverskoe1                     5444837
1952Vostochnoe Izmajlovo2      2002941
1960Izmajlovo2                 7000000
1960Koptevo2                   5286363
1960Lomonosovskoe2            13967753
1960Timirjazevskoe2           10703571
1963Cheremushki5               6740909
1964Caricyno1                  5443181
1965Perovo1                    4961250
1967Kon'kovo1                  7264705
1967Severnoe Tushino1          7305882
1968Hovrino1                   5661666
1968Novogireevo1               6940909
1969Kon'kovo1                  7721739
1969Pechatniki1                5800000
1969Severnoe Tushino1          6684285
1970Severnoe Tushino1          6084615
1970Veshnjaki1                 6210000
1971Veshnjaki1                 6355000
1972Kon'kovo1                  8583333
1972Veshnjaki1                 5692029
1973Veshnjaki1                 5575000
1974Bibirevo1                  6614545
1975Bibirevo1                  7800952
1976Cheremushki1               9362500
1976Zjablikovo1 

In [59]:
col_train=['floor','full_sq','life_sq','max_floor','num_room','price_doc','area_m','material']
col_maxfloor_train = ['floor','full_sq','life_sq','price_doc','area_m']
df_train = df.loc[:,col_train].dropna()
df_maxfloor = df_train.loc[:,col_maxfloor_train]

In [119]:
from sklearn.model_selection import train_test_split
X_train_maxfloor, X_test_maxfloor, y_train_maxfloor, y_test_maxfloor = train_test_split(df_maxfloor.values,df_train.max_floor.values)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_maxfloor)
StandardScaler(copy=True, with_mean=True, with_std=True)
X_train_maxfloor = scaler.transform(X_train_maxfloor)
X_test_maxfloor = scaler.transform(X_test_maxfloor)

In [134]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes = (30,),solver='sgd',learning_rate_init=0.01,max_iter=500,activation='tanh')

mlp.fit(X_train_maxfloor, y_train_maxfloor)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(30,), learning_rate='constant',
       learning_rate_init=0.01, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [135]:
float(np.sum(mlp.predict(X_test_maxfloor) == y_test_maxfloor))/float(len(X_test_maxfloor))

0.38663484486873506

In [126]:
df_maxfloor_pred = df[pd.isnull(df.max_floor)].loc[:,col_maxfloor_train].dropna()

In [127]:
pred_maxfloor = mlp.predict(X_test_maxfloor)